In [14]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import yaml
import os

with open("config.yml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile)

#import data
bulkdir = "Data/Slideseq/" + cfg['default']['slideseqfolder'] + "/results/Bulk"
X = pd.read_csv(os.path.join(bulkdir, "X_bulk.csv"))
b = pd.read_csv(os.path.join(bulkdir, "b_bulk.csv"))
cell_type_names = X.columns[1:]
X = X.iloc[:,1:].to_numpy()
b = b.iloc[:,1].to_numpy()

/Users/dcable/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [15]:
#define the model
class Model(object):
  def __init__(self):
    self.N = len(cell_type_names)
    self.w = tf.Variable([1/self.N for i in range(self.N)])

  def __call__(self, x):
    return tf.tensordot(x[0:(self.N)],tf.abs(self.w),1)
    
  def predict_batch(self, x):
    return tf.linalg.matvec(x[:,0:(self.N)],tf.abs(self.w)) 

def loss(predicted_y, target_y):
  return tf.math.reduce_sum(tf.square(tf.math.log(predicted_y) - tf.math.log(target_y)))

model = Model()

In [16]:
#create the loss function
X_var = tf.compat.v1.placeholder("float32") 
Y_var = tf.compat.v1.placeholder("float32") 
my_loss = loss(model(X_var),Y_var)
batch_loss = loss(model.predict_batch(X_var),Y_var)
optimizer = tf.train.AdamOptimizer(1e-6).minimize(my_loss)
init = tf.global_variables_initializer() 


In [20]:
#train the model
training_epochs = 2000
with tf.Session() as sess: 
     
    # Initializing the Variables 
    sess.run(init) 
      
    # Iterating through all the epochs 
    for epoch in range(training_epochs): 
          
        # Feeding each data point into the optimizer using Feed Dictionary 
        for (_x, _y) in zip(X, b): 
            sess.run(optimizer, feed_dict = {X_var : _x, Y_var : _y}) 
          
        # Displaying the result after every 5 epochs 
        if (epoch + 1) % 20 == 0: 
            # Calculating the cost a every epoch 
            c = sess.run(batch_loss, feed_dict = {X_var : X, Y_var : b}) 
            print("Epoch", (epoch + 1), ": cost =", c, "w =", sess.run(model.w)) 
      
    # Storing necessary values to be used outside the Session 
    training_cost = sess.run(batch_loss, feed_dict = {X_var : X, Y_var : b}) 
    weight = sess.run(model.w) 


Epoch 20 : cost = 2600.0908 w = [0.04692438 0.04771224 0.04797851 0.048215   0.04499157 0.05069632
 0.04634752 0.04615641 0.04620568 0.04655871 0.04528831 0.04700816
 0.04644945 0.05374171 0.05425942 0.04790708 0.04671956 0.04671363
 0.05110411 0.04864939]
Epoch 40 : cost = 2458.958 w = [0.04377104 0.04539571 0.04596221 0.04648725 0.0399751  0.05144321
 0.04275548 0.04241748 0.04258171 0.04326438 0.04063711 0.04428412
 0.04295509 0.05746626 0.05844798 0.0458765  0.0436294  0.04366722
 0.05223867 0.0473196 ]
Epoch 60 : cost = 2317.8762 w = [0.04065181 0.04311668 0.04398932 0.04484223 0.03505027 0.05219228
 0.0392316  0.03875503 0.03907611 0.04003782 0.03607005 0.04171798
 0.03948853 0.06116783 0.06261452 0.04388267 0.04063789 0.04074243
 0.05336514 0.04598605]
Epoch 80 : cost = 2176.8955 w = [0.03757114 0.04089515 0.04206666 0.04329121 0.03023501 0.05294886
 0.03578869 0.03518412 0.03571765 0.03689016 0.03160179 0.03932383
 0.03605488 0.06484699 0.06675879 0.04194557 0.03775851 0.037956

Epoch 600 : cost = 788.7925 w = [-1.1229973e-05  2.6753113e-02  1.7371282e-02  3.5028752e-02
  3.8131585e-03  8.0449544e-02  4.0066917e-03  8.0309538e-03
  2.1975614e-02  3.6926912e-03  1.3958323e-03  3.1197732e-02
  3.0547195e-05  1.5865625e-01  1.6454384e-01  3.0681036e-02
  1.3565688e-02  1.6214307e-02  9.6296705e-02  2.4984084e-02]
Epoch 620 : cost = 783.68604 w = [-8.5656002e-06  2.6906347e-02  1.6926160e-02  3.4979343e-02
  3.8661631e-03  8.1291974e-02  3.9206743e-03  8.0626663e-03
  2.2181256e-02  3.3969008e-03  1.3421030e-03  3.1395875e-02
  3.1273012e-05  1.6191064e-01  1.6770421e-01  3.0587075e-02
  1.3456555e-02  1.6076367e-02  9.7904921e-02  2.4347050e-02]
Epoch 640 : cost = 778.85986 w = [-7.8663907e-06  2.7047424e-02  1.6493838e-02  3.4920435e-02
  3.9030297e-03  8.2095869e-02  3.8430640e-03  8.0921799e-03
  2.2370243e-02  3.1190040e-03  1.2902153e-03  3.1583801e-02
  9.1837774e-06  1.6511652e-01  1.7081088e-01  3.0477470e-02
  1.3355129e-02  1.5944207e-02  9.9482588e-02 

Epoch 1080 : cost = 718.2084 w = [-1.21437461e-05  2.86494121e-02  1.07979001e-02  3.31410095e-02
  3.83417006e-03  9.27604586e-02  3.27002699e-03  8.30935035e-03
  2.40083821e-02  2.51468591e-04  8.16008018e-04  3.41587104e-02
 -2.40408144e-06  2.24657759e-01  2.26818666e-01  2.64587179e-02
  1.21545475e-02  1.42574720e-02  1.28225908e-01  1.21394992e-02]
Epoch 1100 : cost = 716.69965 w = [-1.1709325e-05  2.8675616e-02  1.0679248e-02  3.3071179e-02
  3.8229136e-03  9.3011662e-02  3.2633040e-03  8.3065089e-03
  2.4015805e-02  2.0427327e-04  8.0618542e-04  3.4227371e-02
 -2.1697913e-06  2.2691049e-01  2.2886355e-01  2.6257606e-02
  1.2123913e-02  1.4218093e-02  1.2930712e-01  1.1762837e-02]
Epoch 1120 : cost = 715.14557 w = [ 3.9525476e-06  2.8698424e-02  1.0568241e-02  3.3002809e-02
  3.8117159e-03  9.3246967e-02  3.2569596e-03  8.3034979e-03
  2.4020888e-02  1.6026673e-04  7.9663302e-04  3.4292880e-02
 -2.1370806e-06  2.2912821e-01  2.3087199e-01  2.6057802e-02
  1.2094326e-02  1.4180

Epoch 1580 : cost = 694.9906 w = [-1.0045791e-05  2.8452242e-02  9.2951637e-03  3.1822979e-02
  3.5985750e-03  9.5501155e-02  3.0212812e-03  8.2121855e-03
  2.3636494e-02  6.4133460e-06  6.2979443e-04  3.5065137e-02
  7.2982943e-06  2.7146292e-01  2.6815426e-01  2.1902096e-02
  1.1615531e-02  1.3699123e-02  1.5092058e-01  6.4241542e-03]
Epoch 1600 : cost = 694.4514 w = [-2.64449977e-06  2.84221601e-02  9.27181635e-03  3.17859650e-02
  3.59187718e-03  9.55001190e-02  3.01039382e-03  8.21073726e-03
  2.36114804e-02  9.01856220e-06  6.24131295e-04  3.50771919e-02
 -2.41477883e-05  2.72971153e-01  2.69444674e-01  2.17460617e-02
  1.16010206e-02  1.36893755e-02  1.51666030e-01  6.31953171e-03]
Epoch 1620 : cost = 694.1538 w = [-1.0788939e-05  2.8390761e-02  9.2498343e-03  3.1749360e-02
  3.5851849e-03  9.5492810e-02  2.9997225e-03  8.2096299e-03
  2.3586571e-02 -8.8028673e-06  6.1852467e-04  3.5088208e-02
  3.4059034e-05  2.7445811e-01  2.7071103e-01  2.1592069e-02
  1.1586949e-02  1.367989

In [21]:
weight_df = pd.DataFrame(abs(weight),index=cell_type_names,columns=["Weight"])
weight_df.to_csv(os.path.join(bulkdir, "weights.csv"))


In [22]:
sum(abs(weight))

1.039219340088266